In [12]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.utils.np_utils import to_categorical
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam

train_df = pd.read_json('../input/train.json')
test_df = pd.read_json('../input/test.json')

In [13]:
def std_img(x):
    for i in range(3):
        x[:, :, i] -= np.mean(x[:, :, i].flatten())
        x[:, :, i] /= np.std(x[:, :, i].flatten()) + 1e-7
    return x

def get_image(df):
    '''Create 3-channel 'images'. Return rescale-normalised images.'''
    images = []
    for i, row in df.iterrows():
        # Formulate the bands as 75x75 arrays
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = (band_1 + band_2)/2

        # Rescale
        img = np.dstack([band_1,band_2,band_3])
        img = std_img(img)

        images.append(img)
    return np.array(images)


train_x = get_image(train_df)
test_x = get_image(test_df)

print(train_x.shape,test_x.shape)

(1604, 75, 75, 3) (8424, 75, 75, 3)


In [14]:
y = train_df.is_iceberg.values
print(y[:5])

[0 0 1 0 0]


In [15]:
from keras.callbacks import ModelCheckpoint,LearningRateScheduler
def create_model():
    '''Create the FCN and return a keras model.'''

    model = Sequential()
    model.add(Conv2D(32, (3, 3),input_shape=(75, 75, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(32, (3, 3), activation='relu', strides=1))
    model.add(Conv2D(32, (3, 3), activation='relu', strides=1))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(64, (3, 3), activation='relu', strides=1))
    model.add(Conv2D(64, (3, 3), activation='relu', strides=1))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(Conv2D(64, (3, 3), activation='relu', strides=1))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(1, activation='sigmoid'))
    return model
print('model model')


model model


In [17]:
from sklearn.model_selection import KFold
from keras.preprocessing.image import ImageDataGenerator

def lr_f(epoch):
    if epoch<10:
        return 0.001
    elif epoch<30:
        return 0.0005
    else:
        return 0.0001

def kfold_train(fold_cnt=3,rnd=233):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            #zoom_range=0.05,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 32
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=50, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=4)

Epoch 1/50
Epoch 00001: val_loss improved from inf to 0.62132, saving model to best_m.h5
 - 2s - loss: 0.6836 - acc: 0.5429 - val_loss: 0.6213 - val_acc: 0.5337
Epoch 2/50
Epoch 00002: val_loss did not improve
 - 1s - loss: 0.5676 - acc: 0.7099 - val_loss: 0.7373 - val_acc: 0.5337
Epoch 3/50
Epoch 00003: val_loss improved from 0.62132 to 0.59702, saving model to best_m.h5
 - 1s - loss: 0.5538 - acc: 0.7266 - val_loss: 0.5970 - val_acc: 0.6933
Epoch 4/50
Epoch 00004: val_loss improved from 0.59702 to 0.42831, saving model to best_m.h5
 - 1s - loss: 0.4827 - acc: 0.7894 - val_loss: 0.4283 - val_acc: 0.8229
Epoch 5/50
Epoch 00005: val_loss improved from 0.42831 to 0.38927, saving model to best_m.h5
 - 1s - loss: 0.4236 - acc: 0.8217 - val_loss: 0.3893 - val_acc: 0.8329
Epoch 6/50
Epoch 00006: val_loss improved from 0.38927 to 0.38241, saving model to best_m.h5
 - 1s - loss: 0.4232 - acc: 0.8308 - val_loss: 0.3824 - val_acc: 0.8279
Epoch 7/50
Epoch 00007: val_loss improved from 0.38241 to 

Epoch 12/50
Epoch 00012: val_loss did not improve
 - 1s - loss: 0.2475 - acc: 0.8947 - val_loss: 0.3110 - val_acc: 0.8653
Epoch 13/50
Epoch 00013: val_loss improved from 0.29828 to 0.28411, saving model to best_m.h5
 - 1s - loss: 0.2736 - acc: 0.8896 - val_loss: 0.2841 - val_acc: 0.8828
Epoch 14/50
Epoch 00014: val_loss did not improve
 - 1s - loss: 0.2586 - acc: 0.8910 - val_loss: 0.3052 - val_acc: 0.8554
Epoch 15/50
Epoch 00015: val_loss did not improve
 - 1s - loss: 0.2574 - acc: 0.8840 - val_loss: 0.2990 - val_acc: 0.8828
Epoch 16/50
Epoch 00016: val_loss improved from 0.28411 to 0.28220, saving model to best_m.h5
 - 1s - loss: 0.2158 - acc: 0.9138 - val_loss: 0.2822 - val_acc: 0.8853
Epoch 17/50
Epoch 00017: val_loss did not improve
 - 1s - loss: 0.2269 - acc: 0.9028 - val_loss: 0.2938 - val_acc: 0.8903
Epoch 18/50
Epoch 00018: val_loss did not improve
 - 1s - loss: 0.2527 - acc: 0.8919 - val_loss: 0.3602 - val_acc: 0.8429
Epoch 19/50
Epoch 00019: val_loss did not improve
 - 1s - 

Epoch 23/50
Epoch 00023: val_loss did not improve
 - 1s - loss: 0.2232 - acc: 0.9082 - val_loss: 0.3821 - val_acc: 0.8404
Epoch 24/50
Epoch 00024: val_loss did not improve
 - 1s - loss: 0.2737 - acc: 0.8716 - val_loss: 0.2632 - val_acc: 0.8678
Epoch 25/50
Epoch 00025: val_loss did not improve
 - 1s - loss: 0.2126 - acc: 0.9099 - val_loss: 0.2426 - val_acc: 0.9302
Epoch 26/50
Epoch 00026: val_loss did not improve
 - 1s - loss: 0.2443 - acc: 0.8871 - val_loss: 0.2784 - val_acc: 0.8653
Epoch 27/50
Epoch 00027: val_loss improved from 0.24231 to 0.23987, saving model to best_m.h5
 - 1s - loss: 0.1965 - acc: 0.9088 - val_loss: 0.2399 - val_acc: 0.9102
Epoch 28/50
Epoch 00028: val_loss improved from 0.23987 to 0.21942, saving model to best_m.h5
 - 1s - loss: 0.2286 - acc: 0.9020 - val_loss: 0.2194 - val_acc: 0.9077
Epoch 29/50
Epoch 00029: val_loss did not improve
 - 1s - loss: 0.2238 - acc: 0.8980 - val_loss: 0.2745 - val_acc: 0.8628
Epoch 30/50
Epoch 00030: val_loss did not improve
 - 1s - 

Epoch 35/50
Epoch 00035: val_loss improved from 0.24613 to 0.23854, saving model to best_m.h5
 - 1s - loss: 0.2268 - acc: 0.9104 - val_loss: 0.2385 - val_acc: 0.8978
Epoch 36/50
Epoch 00036: val_loss did not improve
 - 1s - loss: 0.2063 - acc: 0.9242 - val_loss: 0.2538 - val_acc: 0.8853
Epoch 37/50
Epoch 00037: val_loss improved from 0.23854 to 0.23847, saving model to best_m.h5
 - 1s - loss: 0.2111 - acc: 0.9122 - val_loss: 0.2385 - val_acc: 0.8903
Epoch 38/50
Epoch 00038: val_loss did not improve
 - 1s - loss: 0.2149 - acc: 0.9166 - val_loss: 0.2394 - val_acc: 0.8878
Epoch 39/50
Epoch 00039: val_loss improved from 0.23847 to 0.23481, saving model to best_m.h5
 - 1s - loss: 0.2185 - acc: 0.9116 - val_loss: 0.2348 - val_acc: 0.9002
Epoch 40/50
Epoch 00040: val_loss did not improve
 - 1s - loss: 0.2166 - acc: 0.9037 - val_loss: 0.2442 - val_acc: 0.8903
Epoch 41/50
Epoch 00041: val_loss did not improve
 - 1s - loss: 0.2023 - acc: 0.9166 - val_loss: 0.2518 - val_acc: 0.8803
Epoch 42/50
Ep

In [18]:
import pickle
with open('../features/cnn_2_aug1_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
from sklearn.metrics import log_loss
print(log_loss(y,train_pred))
    
submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/cnn_2_aug1_sub.csv', index=False)

0.224945227057
         id  is_iceberg
0  5941774d    0.023346
1  4023181e    0.616447
2  b20200e4    0.083889
3  e7f018bb    0.996865
4  4371c8c3    0.004637
